<a href="https://colab.research.google.com/github/kapitalistka/SberSchoolGroupProject/blob/main/Masloed_attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/c/ieee-fraud-detection/

Детекция мошеннических операций. Предсказание мошенничества крайне актуальная задача в современных финансовых организациях. Обратите внимание на несбалансированность классов. Это может стать большой проблемой при решении.


Categorical Features 
- Transaction
- ProductCD
-card1 
- card6
-addr1, addr2
-P_emaildomain
-R_emaildomain
-M1 - M9


Categorical Features 
- Identity
-DeviceType
-DeviceInfo
-id_12 - id_38


The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).

In [1]:
#!pip install kaggle
from google.colab import files
from google.colab import drive
from zipfile import ZipFile
from google.colab import files

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
drive.mount('/content/gdrive')
pd.options.display.float_format = "{:,.2f}".format

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
def update_kaggle_token():
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  
  # Then move kaggle.json into the folder where the API expects to find it.
  !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

update_kaggle_token()

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [3]:
%cd /
%cd /content
!mkdir -p kaggle/GP2/
%cd kaggle/GP2
!kaggle competitions download -c ieee-fraud-detection
%cd /content


/
/content
/content/kaggle/GP2
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
/content


In [4]:
path = "kaggle/GP2/"
file_train_transactions = "train_transaction.csv"
file_test_transactions = "test_transaction.csv"
file_train_identity = "train_identity.csv"
file_test_identity = "test_identity.csv"

In [5]:
def get_zip_file(path, file_name):
  return path + file_name + ".zip"

def get_big_data_from_zip(path, file_name):
  file = ZipFile(get_zip_file(path, file_name)).open(file_name)
  tp = pd.read_csv(file, iterator=True, chunksize=10_000)
  return pd.concat(tp)


In [6]:
%time raw_train_transactions = get_big_data_from_zip(path, file_train_transactions)

%time raw_test_transactions = get_big_data_from_zip(path, file_test_transactions)

%time raw_train_identity = get_big_data_from_zip(path, file_train_identity)

%time raw_test_identity = get_big_data_from_zip(path, file_test_identity)

CPU times: user 23.2 s, sys: 2.26 s, total: 25.5 s
Wall time: 25.5 s
CPU times: user 20.9 s, sys: 899 ms, total: 21.8 s
Wall time: 21.8 s
CPU times: user 847 ms, sys: 11.9 ms, total: 859 ms
Wall time: 867 ms
CPU times: user 805 ms, sys: 19.1 ms, total: 825 ms
Wall time: 829 ms


In [9]:
 raw_train_transactions.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [10]:
 raw_train_identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

In [8]:
print(f"train_transactions {raw_train_transactions.shape}")
print(f"train_identity {raw_train_identity.shape}")
print(f"test_transactions {raw_test_transactions.shape}")
print(f"test_identity {raw_test_identity.shape}")

train_transactions (590540, 394)
train_identity (144233, 41)
test_transactions (506691, 393)
test_identity (141907, 41)


In [15]:
train = pd.merge(raw_train_transactions, raw_train_identity, how = 'left', on = 'TransactionID')
print(train.shape)

test = pd.merge(raw_test_transactions, raw_test_identity, how = 'left', on = 'TransactionID')
print(test.shape)

(590540, 434)
(506691, 433)


In [14]:
train.head()


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,nan,150.00,discover,142.00,credit,315.00,87.00,19.00,nan,NaN,NaN,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,2.00,0.00,1.00,1.00,14.00,nan,13.00,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.00,150.00,mastercard,102.00,credit,325.00,87.00,nan,nan,gmail.com,NaN,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,0.00,nan,nan,0.00,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.00,150.00,visa,166.00,debit,330.00,87.00,287.00,nan,outlook.com,NaN,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,nan,nan,0.00,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.00,150.00,mastercard,117.00,debit,476.00,87.00,nan,nan,yahoo.com,NaN,2.00,5.00,0.00,0.00,0.00,4.00,0.00,0.00,1.00,0.00,1.00,0.00,25.00,1.00,112.00,112.00,0.00,94.00,0.00,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.00,150.00,mastercard,102.00,credit,420.00,87.00,nan,nan,gmail.com,NaN,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,1.00,1.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan,...,0.00,"70,787.00",nan,nan,nan,nan,nan,nan,nan,nan,100.00,NotFound,nan,-480.00,New,NotFound,166.00,nan,542.00,144.00,nan,nan,NaN,nan,nan,nan,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.00,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
